In [1]:
import pandas as pd
import numpy as np
from torcheval.metrics.functional import binary_precision_recall_curve
import torch
from proteinfertorch.utils import read_pickle, read_fasta,save_to_fasta, load_emeddings, generate_vocabularies
import matplotlib.pyplot as plt
import os


/home/samirchar/miniconda3/envs/proteinfertorch_310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
rank=1
{"cuda:%d" % 0: "cuda:%d" % rank}

{'cuda:0': 'cuda:1'}

In [ ]:

# Load the entire checkpoint
map_location = {"cuda:%d" % 0: "cuda:%d" % 0}
checkpoint = torch.load(
    '../data/checkpoints/go-random_2024-12-22_19-57-30_checkpoint_epoch_59_2024-12-22_19-57-30.pt',
    map_location=map_location
    )

/tmp/ipykernel_365864/2049962199.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('../data/checkpoints/go-random_2024-12-22_19-57-30_checkpoint_ep

In [23]:
checkpoint['model_state_dict'].keys()

odict_keys(['conv1.weight', 'conv1.bias', 'resnet_blocks.0.bn_activation_1.0.weight', 'resnet_blocks.0.bn_activation_1.0.bias', 'resnet_blocks.0.bn_activation_1.0.running_mean', 'resnet_blocks.0.bn_activation_1.0.running_var', 'resnet_blocks.0.bn_activation_1.0.num_batches_tracked', 'resnet_blocks.0.masked_conv1.weight', 'resnet_blocks.0.masked_conv1.bias', 'resnet_blocks.0.bn_activation_2.0.weight', 'resnet_blocks.0.bn_activation_2.0.bias', 'resnet_blocks.0.bn_activation_2.0.running_mean', 'resnet_blocks.0.bn_activation_2.0.running_var', 'resnet_blocks.0.bn_activation_2.0.num_batches_tracked', 'resnet_blocks.0.masked_conv2.weight', 'resnet_blocks.0.masked_conv2.bias', 'resnet_blocks.1.bn_activation_1.0.weight', 'resnet_blocks.1.bn_activation_1.0.bias', 'resnet_blocks.1.bn_activation_1.0.running_mean', 'resnet_blocks.1.bn_activation_1.0.running_var', 'resnet_blocks.1.bn_activation_1.0.num_batches_tracked', 'resnet_blocks.1.masked_conv1.weight', 'resnet_blocks.1.masked_conv1.bias', 'res

In [ ]:
from proteinfertorch.proteinfer import ProteInfer

model = ProteInfer().to('cuda:0')
load_model(model, optimizer, checkpoint_path: str, rank: int)

In [ ]:
def load_checkpoint(model,
               optimizer,
               checkpoint_path: str,
               rank: int,
               lr_scheduler=None):
    """
    Load the model's state from a given checkpoint.

    Note:
    The function assumes that the model in the trainer object is NOT DDP-wrapped.
    """
    torch.cuda.empty_cache()

    # Load the entire checkpoint
    map_location = {"cuda:%d" % 0: "cuda:%d" % rank}
    checkpoint = torch.load(checkpoint_path, map_location=map_location)

    # Extract the state_dict from the checkpoint
    state_dict = checkpoint["model_state_dict"]

    # Load the state_dict into the model
    model.load_state_dict(state_dict)

    metadata = {}
    # Load the optimizer state and epoch number if they exist in the checkpoint
    if checkpoint["optimizer_state_dict"]:
        optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
    if checkpoint["epoch"]:
        metadata.update(checkpoint["epoch"])
    if checkpoint["validation_metrics"]:
        metadata.update(checkpoint["validation_metrics"])
    if checkpoint["train_metrics"]:
        metadata.update(checkpoint["train_metrics"])
    if checkpoint["lr_scheduler_state_dict"]:
        lr_scheduler.load_state_dict(checkpoint["lr_scheduler_state_dict"])
    

    return model, optimizer, lr_scheduler, metadata

In [2]:
from proteinfertorch.utils import ReadFasta,get_huggingface_url
hf_url = get_huggingface_url(repo_id="samirchar/proteinfer_data",data_path="random_split/test_EC.fasta")
data = ReadFasta(data_path=hf_url)()

FileNotFoundError: [Errno 2] No such file or directory: '../data/random_split/dev_EC.fasta'

In [8]:
a

([('MLKNDLFLRALKRQPCSRTPIWVMRQAGRYLPEYRAVREKTDFLTLCKTPELATEVTIQPVELVGVDAAIIFSDILVVNEAMGQEVNIIETKGIKLAPPIRSQADIDKLIVPDIDEKLGYVLDALRMTKKELDNRVPLIGFSGAAWTLFTYAVEGGGSKNYAYAKQMMYREPQMAHSLLSKISQTITAYTLKQIEAGADAIQIFDSWASALSEDDYREYALPYIKDTVQAIKAKHPETPVIVFSKDCNTILSDIADTGCDAVGLGWGIDISKARTELNDRVALQGNLDPTVLYGTQERIKIEAGKILKSFGQHNHHSGHVFNLGHGILPDMDPDNLRCLVEFVKEESAKYH',
   'Q3AUB7',
   ['EC:4.-.-.-', 'EC:4.1.1.37', 'EC:4.1.-.-', 'EC:4.1.1.-']),
  ('MKLILLILTAYLLGSIPTGLWIGQYFYNINLREHGSGNTGTTNTFRILGLKAGAATLLIDIFKGTLATLLPVLVGASNVSPIAIGFFAVLGHTFPIFAGFKGGKAVATSAGVLLGFAPLYLLFLAAVFVLTLYLFSMISLASLTASVVAVISVLTFPAAHFLLPGYDWLLTITIVVLAAIIILRHQDNMKRIKQQSENLIPWGLNLSKQQPASHH',
   'B4U3D2',
   ['EC:2.3.1.-', 'EC:2.3.1.275', 'EC:2.-.-.-', 'EC:2.3.-.-']),
  ('MAVHNLKDLLAEGVSGRGVLVRSDLNVPLDSDGEQGRITDPGRITASVPTLSALVEAGAKVVVAAHLGRPKNGPDPALSLAPVAAALGEQLGRHVQLASDVVGTDALARAEGLTDGDVLLLENIRFDARETSKDDAERLALARQLAELVGPTGAFVSDGFGVVHRKQASVYDVATLLPHYAGTLVAEEIAVLEQLTGSTKRPYAVVLGGSKVSDKLGVIESLATKADSIVIGGGMCFTFLAAQGFSVGKSLLETEMVDTCRRL

In [2]:
import requests
from io import StringIO
response = requests.get("www.huggingface.co/datasets/samirchar/proteinfer_data/resolve/main/random_split/test_EC.fasta")
data_path = StringIO(response.text)
test_EC,has_labels = read_fasta(data_path)


MissingSchema: Invalid URL 'www.huggingface.co/datasets/samirchar/proteinfer_data/resolve/main/random_split/test_EC.fasta': No scheme supplied. Perhaps you meant https://www.huggingface.co/datasets/samirchar/proteinfer_data/resolve/main/random_split/test_EC.fasta?

In [9]:
has_labels

True

In [ ]:
(response.text[0:10])

'>Q3AUB7 EC'

In [13]:
from datasets import DatasetDict, load_dataset
from huggingface_hub import login, ModelCard
card = ModelCard(
    """

    """
    )

# data_files = {"dev":'dev_EC.fasta', "test":'test_EC.fasta'}
data_files = {"dev":'test_csv.csv', "test":'test_csv.csv'}
d = load_dataset('audiofolder',data_dir="../data/random_split/",data_files=data_files)
d.push_to_hub()


Repo card metadata block was not found. Setting CardData to empty.
Generating dev split: 0 examples [00:00, ? examples/s]
Generating test split: 0 examples [00:00, ? examples/s]


ValueError: Instruction "dev" corresponds to no data!

In [7]:
load_emeddings('../outputs/embeddings/').shape

/home/samirchar/proteinfertorch/proteinfertorch/utils.py:489: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  embeddings.append(torch.load(f"{dir}/{partitions[partition_idx]}"

torch.Size([5, 1100])

In [7]:
full_go_rand = generate_vocabularies(read_fasta('../data/random_split/full_GO.fasta')[0])
full_ec_rand = generate_vocabularies(read_fasta('../data/random_split/train_GO.fasta')[0])

In [ ]:
len(full_go_rand['label_vocab']), len(full_ec_rand['label_vocab'])

(32102, 31365)

: 

In [14]:
len(full_go_rand['label_vocab']), len(full_ec_rand['label_vocab']), len(full_go_clust['label_vocab']), len(full_ec_clust['label_vocab'])

(32102, 5135, 32102, 5135)

In [18]:
full_go_rand['label_vocab'] == full_go_clust['label_vocab'], full_ec_rand['label_vocab']== full_ec_clust['label_vocab']

(True, True)

In [17]:
full_go_rand['label_vocab'] ^ len(full_go_clust['label_vocab'])

TypeError: unsupported operand type(s) for ^: 'list' and 'int'

In [12]:
emb_10 = load_emeddings("../outputs/embeddings_10")
emb_1 = load_emeddings("../outputs/embeddings_1")

/home/samirchar/proteinfertorch/proteinfertorch/utils.py:485: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  embeddings.append(torch.load(f"{dir}/{partitions[partition_idx]}"

In [2]:
ds = []
for split in ["train","dev","test"]:
    data = read_fasta(f"../data/random_split/{split}_GO.fasta")    
    d = [(obs[0],obs[1],obs[2][:2]) for obs in data[:40*8*5]]
    ds.append(d)
    save_to_fasta(d,f"../data/random_split/{split}_GO_mini.fasta")

Saved FASTA file to ../data/random_split/train_GO_mini.fasta
Saved FASTA file to ../data/random_split/dev_GO_mini.fasta
Saved FASTA file to ../data/random_split/test_GO_mini.fasta


In [37]:
from collections import defaultdict
model_ids = defaultdict(list)
file_name2var_name = {
    "noxpd2_cnn_swissprot_go_clustered_swiss-cnn_for_swissprot_go_clustered":"GO_CLUSTERED_ENSEMBLE_ELEMENT_EXPERIMENT_IDS",
    "noxpd2_cnn_swissprot_go_random_swiss-cnn_for_swissprot_go_random":"GO_RANDOM_ENSEMBLE_ELEMENT_EXPERIMENT_IDS",
    "noxpd2_cnn_swissprot_ec_clustered_swiss-cnn_for_swissprot_ec_clustered":"EC_CLUSTERED_ENSEMBLE_ELEMENT_EXPERIMENT_IDS",
    "noxpnd_cnn_swissprot_ec_random_swiss-cnn_for_swissprot_ec_random":"EC_RANDOM_ENSEMBLE_ELEMENT_EXPERIMENT_IDS",

}

for i in pd.read_csv('../zipped_models.txt', header=None).values.flatten():
    file_name = i.split('/')[-1].replace('.tar.gz', '')
    #Only consider file_names with go or ec in them
    if '-'.join(file_name.split('-')[:-1]) in file_name2var_name:
        # Split string noxpd2_cnn_swissprot_ec_clustered_swiss-cnn_for_swissprot_ec_clustered-13704042.tar.gz' by - followed by number and extension
        split = file_name.split('-')
        w_id = int(split[-1])
        name = '-'.join(split[:-1])
        model_ids[file_name2var_name[name]].append(str(w_id))


In [ ]:
labels = pd.read_hdf("../outputs/test_labels_ProteInfer.h5", key="labels_df")
probabilities = pd.read_hdf("../outputs/test_probabilities_ProteInfer.h5", key="probabilities_df")

logits_binary = torch.tensor(probabilities.values.flatten(),device='cuda')
labels_binary = torch.tensor(labels.values.flatten(),device='cuda')
precision, recall, thresholds = binary_precision_recall_curve(logits_binary, labels_binary)
f1 = 2 * precision * recall / (precision + recall)

best_th,best_f1 = thresholds[torch.argmax(f1)].item(),torch.max(f1).item()
print(best_th,best_f1)